In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import path
import glob
sns.set_style('white')

In [8]:
sample_list = pd.read_csv("/Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/sample_list.csv",sep=",",header=0)

am_path= "/Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/analysis3/annotation_measurements/"
dm_path= "/Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/analysis3/detection_measurements/"

figure_outputs = "/Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/results/"
sample_list[:5]


,Sample #,Genotype,Condition,n,date
0,1,HML-GFP,C,8,9/27/2020
1,2,HML-GFP,C,7,9/27/2020
2,3,roGFP,C,10,9/27/2020
3,4,roGFP,C,9,9/27/2020
4,5,drpr∆5,C,7,9/27/2020


In [13]:
## Create a dataframe of annotation measurements (abdomen area and centroid) for all images

def get_sample_condition(imagename,sample_mapping):
    sampleid = imagename[:2]
    condition = sample_mapping[sample_mapping['Sample #'] == int(sampleid)]
    return condition['Condition'].iloc[0]

all_annotation_measurement_files = glob.glob(path.join(am_path , "*.txt"))
am_list = []
for amf in all_annotation_measurement_files:
    df = pd.read_csv(amf, index_col=None, header=0, sep="\t")
    am_list.append(df)

abdomen_coordinates = pd.concat(am_list, axis=0, ignore_index=True)

## annotate each fly abdomen with experimental group info 
abdomen_coordinates['condition'] = abdomen_coordinates['Image'].apply(get_sample_condition,sample_mapping=sample_list)
#abdomen_coordinates = abdomen_coordinates.loc[abdomen_coordinates["condition"] == "C", "condition"] = 'Fed'
abdomen_coordinates['condition'] = np.where(abdomen_coordinates['condition'] == 'C', 'Fed', abdomen_coordinates['condition'])
abdomen_coordinates['condition'] = np.where(abdomen_coordinates['condition'] == 'S', 'Starved', abdomen_coordinates['condition'])

abdomen_coordinates.columns = abdomen_coordinates.columns.str.replace('Centroid X µm', 'Centroid_X')
abdomen_coordinates.columns = abdomen_coordinates.columns.str.replace('Centroid Y µm', 'Centroid_Y')
print(abdomen_coordinates.shape)
abdomen_coordinates[:5]

(212, 14)


,Image,Name,Class,Parent,ROI,Centroid_X,Centroid_Y,Num Detections,Num Positive,Positive %,Num Positive per mm^2,Area µm^2,Perimeter µm,condition
0,75C_6_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,441.41,350.34,38.0,38.0,100.0,210.03,180922.8,1648.00,Fed
1,75C_5_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,458.52,339.08,19.0,19.0,100.0,124.30,152858.2,1528.90,Fed
2,86F_6_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,520.88,302.79,85.0,85.0,100.0,1547.10,54942.6,961.95,Starved
3,75F_1_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,425.08,257.83,13.0,13.0,100.0,111.31,116789.2,1416.40,Fed
4,75E_6_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,441.08,241.79,16.0,16.0,100.0,135.33,118229.6,1434.70,Fed


In [30]:
from scipy.spatial import distance
import math 
sns.set_palette("pastel")

def zero_scale_x(image, x, polygon):
   polygon_x = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_X'].iloc[0] 
   zero_scaled_x = x-polygon_x
   return(zero_scaled_x)

def zero_scale_y(image, y, polygon):
   polygon_y = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_Y'].iloc[0] 
   zero_scaled_y = y-polygon_y
   return(zero_scaled_y)

def get_centroid_distance(image,x,y,polygon):
    polygon_x = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_X'].iloc[0]
    polygon_y = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_Y'].iloc[0]
    centroid_distance = distance.euclidean([polygon_x, polygon_y], [x,y])

    return centroid_distance
    

all_detection_measurement_files = glob.glob(path.join(dm_path , "*.txt"))
dm_list = []

for dm in all_detection_measurement_files:
    #print(dm)
    detection_measurements = pd.read_csv(dm,sep="\t",header=0)
    detection_measurements.columns = detection_measurements.columns.str.replace('Centroid X µm', 'Centroid_X')
    detection_measurements.columns = detection_measurements.columns.str.replace('Centroid Y µm', 'Centroid_Y')
    detection_measurements["centroid_x_zeroscaled"] = detection_measurements.apply(
        lambda x: zero_scale_x(x.Image, x.Centroid_X,polygon=abdomen_coordinates), axis=1) 
    detection_measurements["centroid_y_zeroscaled"] = detection_measurements.apply(
        lambda x: zero_scale_y(x.Image, x.Centroid_Y,polygon=abdomen_coordinates), axis=1)
    detection_measurements["centroid_distances"] = detection_measurements.apply(
        lambda x: get_centroid_distance(x.Image, x.Centroid_X,x.Centroid_Y,polygon=abdomen_coordinates), axis=1)
    dm_list.append(detection_measurements)
    
    
    
#print(dm_list)
hemocyte_coordinates = pd.concat(dm_list, axis=0, ignore_index=True)



In [31]:
hemocyte_coordinates[:5]

,Image,Name,Class,Parent,ROI,Centroid_X,Centroid_Y,Nucleus: Area,Nucleus: Perimeter,Nucleus: Circularity,...,Delaunay: Num neighbors,Delaunay: Mean distance,Delaunay: Median distance,Delaunay: Max distance,Delaunay: Min distance,Delaunay: Mean triangle area,Delaunay: Max triangle area,centroid_x_zeroscaled,centroid_y_zeroscaled,centroid_distances
0,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,431.16,152.82,5.4900,8.0218,1.0000,...,5.0,114.7340,81.8370,351.0382,15.9458,2145.0256,5813.8784,-10.25,-197.52,197.785775
1,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,444.58,156.58,11.3722,14.1223,0.7165,...,5.0,25.0827,15.9458,75.4955,4.9826,155.4801,572.5013,3.17,-193.76,193.785930
2,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,452.20,156.48,19.9993,20.4010,0.6038,...,3.0,8.1604,4.9826,14.9418,4.5568,11.4363,22.9957,10.79,-193.86,194.160047
3,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,465.84,157.54,14.1172,16.4980,0.6518,...,8.0,20.2204,17.4176,45.5685,3.4697,26.0180,64.5575,24.43,-192.80,194.341619
4,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,455.32,163.11,9.4114,12.8552,0.7157,...,6.0,20.7304,11.9790,71.1134,4.5568,121.9538,543.2628,13.91,-187.23,187.746001


In [32]:
hemocyte_coordinates.to_csv("/Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/results/hemocyte_coordinates.csv", sep="\t",index=False)
abdomen_coordinates.to_csv("/Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/results/abdomen_coordinates.csv", sep="\t",index=False)